<a href="https://colab.research.google.com/github/HlalalaEddie1/DataValidation/blob/main/Data_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install great_expectations pandas==2.2.2 numpy==1.26.4 --quiet


In [2]:
!pip install great_expectations

In [3]:
!great_expectations init


  ___              _     ___                  _        _   _
 / __|_ _ ___ __ _| |_  | __|_ ___ __  ___ __| |_ __ _| |_(_)___ _ _  ___
| (_ | '_/ -_) _` |  _| | _|\ \ / '_ \/ -_) _|  _/ _` |  _| / _ \ ' \(_-<
 \___|_| \___\__,_|\__| |___/_\_\ .__/\___\__|\__\__,_|\__|_\___/_||_/__/
                                |_|
             ~ Always know what to expect from your data ~

This looks like an existing project that appears complete! You are ready to roll.



Import Libraries

In [4]:

import pandas as pd
import great_expectations as gx
from pprint import pprint
import json

In [5]:
file_path = '/content/drive/MyDrive/large_bug_reports.csv'
df = pd.read_csv(file_path)
df.head()


,bug_id,summary,severity,bug_type,reported_by
0,1,Slow loading of dashboard,High,Memory,User41
1,2,Button misalignment,Low,Backend,User480
2,3,Memory leak in background job,Low,Crash,User415
3,4,Button misalignment,Medium,Crash,User2
4,5,Button misalignment,Low,Memory,User195


Create Great Expectations context

In [6]:
context = gx.get_context()

Add or overwrite a pandas datasource

In [7]:
try:
    datasource = context.sources.add_pandas(name="bug_reports_pandas")
except gx.exceptions.DataContextError:
    context._project_config["fluent_datasources"].pop("bug_reports_pandas", None)
    datasource = context.sources.add_pandas(name="bug_reports_pandas9")

Register your DataFrame as a data asset

In [8]:
data_asset = datasource.add_dataframe_asset(name="bug_reports_asset")

Build a batch request

In [9]:

batch_request = data_asset.build_batch_request(dataframe=df)

Create or load an expectation suite

In [10]:

suite = context.add_or_update_expectation_suite("bug_reports_suite")


Validate the batch against expectations

In [11]:

validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite=suite
)

Add expectations

In [12]:
validator.expect_column_values_to_not_be_null("bug_type")
validator.expect_column_values_to_be_in_set("severity", ["Low", "Medium", "High", "Critical"])

  warnings.warn(



Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

  warnings.warn(



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": false,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_be_in_set",
    "kwargs": {
      "column": "severity",
      "value_set": [
        "Lowl",
        "Medium",
        "High",
        "Critical"
      ],
      "batch_id": "bug_reports_pandas9-bug_reports_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 100000,
    "unexpected_count": 25154,
    "unexpected_percent": 25.154,
    "partial_unexpected_list": [
      "Low",
      "Low",
      "Low",
      "Low",
      "Low",
      "Low",
      "Low",
      "Low",
      "Low",
      "Low",
      "Low",
      "Low",
      "Low",
      "Low",
      "Low",
      "Low",
      "Low",
      "Low",
      "Low",
      "Low"
    ],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 25.154,
    "unexpected_percent_nonmissing": 25.154
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_

ADDITIONAL EXPECTATIONS (Add Here)

In [13]:
validator.expect_column_values_to_not_be_null("summary")
validator.expect_column_value_lengths_to_be_between("summary", min_value=10, max_value=100)
validator.expect_column_values_to_be_unique("bug_id")
validator.expect_column_values_to_not_be_null("reported_by")
validator.expect_column_values_to_match_regex("reported_by", r"User\d+")

  warnings.warn(



Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

  warnings.warn(



Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

  warnings.warn(



...

  warnings.warn(



Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

  warnings.warn(



Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "expectation_config": {
    "expectation_type": "expect_column_values_to_match_regex",
    "kwargs": {
      "column": "reported_by",
      "regex": "User\\d+",
      "batch_id": "bug_reports_pandas9-bug_reports_asset"
    },
    "meta": {}
  },
  "result": {
    "element_count": 100000,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

Save the suite

In [14]:
validator.save_expectation_suite()

Create and run checkpoint

In [15]:
checkpoint = context.add_or_update_checkpoint(
    name="bug_checkpoint",
    validations=[{
        "batch_request": batch_request,
        "expectation_suite_name": "bug_reports_suite"
    }]
)

Create and run checkpoint

In [16]:

checkpoint = context.add_or_update_checkpoint(
    name="bug_checkpoint",
    validations=[{
        "batch_request": batch_request,
        "expectation_suite_name": "bug_reports_suite"
    }]
)
checkpoint_result = checkpoint.run()

Calculating Metrics:   0%|          | 0/36 [00:00<?, ?it/s]

FAILURE ALERTING (Add Here)

In [17]:
if not checkpoint_result["success"]:
    print(" Data validation failed!")
    for res in checkpoint_result["run_results"].values():
        for evr in res["validation_result"]["results"]:
            if not evr["success"]:
                print(f"FAILED: {evr['expectation_config']['expectation_type']} on column {evr['expectation_config']['kwargs'].get('column')}")


Print result summary

In [18]:
pprint(checkpoint_result.to_json_dict())

{'checkpoint_config': {'action_list': [{'action': {'class_name': 'StoreValidationResultAction'},
                                        'name': 'store_validation_result'},
                                       {'action': {'class_name': 'StoreEvaluationParametersAction'},
                                        'name': 'store_evaluation_params'},
                                       {'action': {'class_name': 'UpdateDataDocsAction'},
                                        'name': 'update_data_docs'}],
                       'batch_request': {},
                       'class_name': 'Checkpoint',
                       'config_version': 1.0,
                       'default_validation_id': None,
                       'evaluation_parameters': {},
                       'expectation_suite_ge_cloud_id': None,
                       'expectation_suite_name': None,
                       'ge_cloud_id': None,
                       'module_name': 'great_expectations.checkpoint',
           

Build and open data docs

In [19]:
context.build_data_docs()
# context.open_data_docs()  # Uncomment this locally (not in Colab)

{'local_site': 'file:///content/gx/uncommitted/data_docs/local_site/index.html'}

Save checkpoint results to file

In [20]:

with open("checkpoint_result.json", "w") as f:
    json.dump(checkpoint_result.to_json_dict(), f, indent=4)


In [21]:
suite_name = "bug_reports_suite"
context.add_or_update_expectation_suite(suite_name)


{
  "expectation_suite_name": "bug_reports_suite",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.18.22"
  }
}